## Setting

In [2]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import shutil
import os

# Optimizer
from torch.optim import Adam


from GPUtil import showUtilization as gpu_usage
from numba import cuda



device = torch.device("cuda:0")

In [3]:
temp = 0.3
loss = 'hmce'
save_folder = './model'

## Seed Fix

In [4]:
import random

SEED  = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Model Load

In [5]:
from transformers import ElectraModel
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [6]:
from transformers import AutoConfig
configuration = AutoConfig.from_pretrained("monologg/koelectra-base-v3-discriminator")
configuration.hidden_dropout_prob = 0.2
configuration.attention_probs_dropout_prob = 0.25

In [7]:
class Encoder(nn.Module):
    def __init__(self, dropout=0.5):
        super(Encoder,self).__init__()
        self.encoder  = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
    def forward(self, input_id, mask):
        x = self.encoder(input_ids = input_id, attention_mask = mask, return_dict = False)
        x = x[0][:,0,:]
        return x 

In [8]:
class aug_Encoder(nn.Module):
    def __init__(self, dropout=0.5):
        super(aug_Encoder,self).__init__()
        self.encoder = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator", config = configuration)
    def forward(self, input_id, mask):
        x = self.encoder(input_ids = input_id, attention_mask = mask, return_dict = False)
        x = x[0][:,0,:]
        return x 

In [9]:
class Classifier(nn.Module):
    """Linear Classifier"""
    def __init__(self, dropout=0.5):
        super(Classifier, self).__init__()
        self.encoder = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.fc = nn.Linear(768,6)
        
    def forward(self,input_id, mask):
        x = self.encoder(input_ids = input_id, attention_mask = mask, return_dict = False)
        x = self.fc(x[0][:,0,:])
        return x

## Dataset

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

train_data = pd.read_excel('Dataset/Training.xlsx')
original_valid_data = pd.read_excel('Dataset/Validation.xlsx')
valid_data, test_data = train_test_split(original_valid_data,test_size=0.5, random_state=42, shuffle =True)

In [11]:
labels = {
    "기쁨":0,
    "불안":1,
    "분노":2,
    "당황":3,
    "슬픔":4,
    "상처":5
}
category = {
    "기쁨":0,
    "불안":1,
    "분노":1,
    "당황":1,
    "슬픔":1,
    "상처":1
}

In [12]:
class Dataset(Dataset):
    def __init__(self,df):
        self.labels = [labels[label] for label in df['감정_대분류']]
        self.category = [category[label] for label in df['감정_대분류']]
        self.texts = [tokenizer(str(sentence), padding = "max_length", max_length=128, truncation=True, return_tensors='pt') for sentence in df['사람문장1']]
        # self.aug_encoder = aug_Encoder()
        
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_data(self,idx):
        return self.texts[idx]
    
    # def get_aug_data(self,idx):
    #     aug_text = self.aug_encoder(self.texts[idx]['input_ids'], self.texts[idx]['attention_mask'])
    #     aug_text = aug_text.squeeze().cuda()
    #     return aug_text

    def get_label(self,idx):
        return np.array(self.labels[idx])
    
    def get_category(self,idx):
        return np.array(self.category[idx])
        
    def __getitem__(self,idx):
        text = self.get_data(idx)
        # aug_text = self.get_aug_data(idx) # Embedded value return
        category = self.get_category(idx)
        label = self.get_label(idx)
        return text, category, label          

## Data Augmentation

### Back Translation

In [13]:
# trans_list = []
# backtrans_list = []

In [14]:
# import selenium
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# import time


# driver = webdriver.Chrome('chromedriver.exe')
# driver.maximize_window()

# def kor_to_trans(text_data, trans_lang):
#     """
#     trans_lang에 넣는 파라미터 값:
#     'en' -> 영어
#     'ja&hn=0' -> 일본어
#     'zh-CN' -> 중국어(간체)
#     """
#     error = 0
#     for i in tqdm(range(len(text_data))):
#         try:
#             driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang+'&st='+text_data[i])
#             # WebDriverWait(driver,10).until(EC.presence_of_element_located(by=By.XPATH,value='//*[@id="txtTarget"]/span'))
#             time.sleep(7)
#             backtrans = driver.find_element(by=By.XPATH,value='//*[@id="txtTarget"]').text
#             trans_list.append(backtrans)
#         except:
#             try:
#                 driver.get('https://papago.naver.com/?sk=ko&tk='+trans_lang)
#                 time.sleep()
#                 driver.find_element(by=By.XPATH,value='//*[@id="txtSource"]').send_keys(text_data[i])
#                 # WebDriverWait(driver,10).until(EC.presence_of_element_located(by=By.XPATH,value='//*[@id="txtTarget"]/span'))
#                 time.sleep(5)
#                 backtrans = driver.find_element(by=By.XPATH,value='//*[@id="txtTarget"]').text
#                 trans_list.append(backtrans)
#             except:
#                 trans_list.append(" ")
#                 error +=1
#                 if error % 10 ==0:
#                     print(error)
#     print(error)

# def trans_to_kor(transed_list, transed_lang):
#     error = 0
#     for i in tqdm(range(len(transed_list))):
#         try:
#             driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko&st='+transed_list[i])
#             # WebDriverWait(driver,10).until(EC.presence_of_element_located(by=By.XPATH,value='//*[@id="txtTarget"]/span'))
#             time.sleep(5)
#             backtrans = driver.find_element(by=By.XPATH,value='//*[@id="txtTarget"]').text
#             backtrans_list.append(backtrans)
#         except:
#             try:
#                 driver.get('https://papago.naver.com/?sk='+transed_lang+'&tk=ko')
#                 time.sleep(5)
#                 driver.find_element(by=By.XPATH,value='//*[@id="txtSource"]').send_keys(transed_list[i])
#                 # WebDriverWait(driver,10).until(EC.presence_of_element_located(by=By.XPATH,value='//*[@id="txtTarget"]/span'))
#                 time.sleep(5)
#                 backtrans = driver.find_element(by=By.XPATH,value='//*[@id="txtTarget"]').text
#                 backtrans_list.append(backtrans)
#             except:
#                 backtrans_list.append(" ")
#                 error+=1
#                 if error % 10 == 0:
#                     print(error)
#     print(error)

In [15]:
# trans_list = []
# backtrans_list = []

In [16]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# import time

# def kor_to_trans(text_data, trans_lang):
#     error = 0
#     chrome_driver = ChromeDriverManager().install()
#     service = Service(chrome_driver)
#     driver = webdriver.Chrome(service=service)


#     for i in tqdm(range(len(text_data))):
#         URL = "https://papago.naver.com/"
#         driver.get(URL)
#         time.sleep(3)
#         question = text_data[i]
#         try:
#             form = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
#             form.send_keys(question)
#             button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
#             button.click()
#             time.sleep(3)

#             result = driver.find_element(By.CSS_SELECTOR, "div#txtTarget")
#             # print(question, "->", result.text)
#             trans_list.append(result)
#         except:
#             trans_list.append(" ")
#             error += 1
#             if (error%10 == 0):
#                 print(error)

# def trans_to_kor(text_data, trans_lang):
#     error = 0
#     chrome_driver = ChromeDriverManager().install()
#     service = Service(chrome_driver)
#     driver = webdriver.Chrome(service=service)

#     for i in tqdm(range(len(text_data))):
#         URL = "https://papago.naver.com/"
#         driver.get(URL)
#         time.sleep(3)
#         question = text_data[i]
#         try:
#             form = driver.find_element(By.CSS_SELECTOR, "textarea#txtSource")
#             form.send_keys(question)

#             button = driver.find_element(By.CSS_SELECTOR, "button#btnTranslate")
#             button.click()
#             time.sleep(3)

#             result = driver.find_element(By.CSS_SELECTOR, "div#txtTarget")
#             # print(question, "->", result.text)
#             backtrans_list.append(result)
#         except:
#             backtrans_list.append(" ")
#             error += 1
#             if (error%10 == 0):
#                 print(error)


In [17]:
# for i in range(520):
#     driver = webdriver.Chrome('chromedriver.exe')
#     trans_list = []
#     backtrans_list = []
#     if i == 519:
#         train_sentence = train_data['사람문장1'][i*100:].tolist()
#     else:
#         train_sentence = train_data['사람문장1'][i*100:(i+1)*100].tolist()
#     kor_to_trans(train_sentence,'en')
#     trans_to_kor(trans_list,'en')
#     backtrans_df = pd.DataFrame(backtrans_list,columns=['증강 문장'])
#     backtrans_df.to_excel('./train_sentence_augmentation{}.xlsx'.format(i))
#     driver.close()

### Dropout base

In [18]:
# sentence  = "나는 행복하다"
# aug_sentence = "나는 행복하다"

# sentence_input = tokenizer(sentence,padding='max_length', max_length = 256, 
#                     truncation=True, return_tensors="pt")
# aug_sentence_input = tokenizer(aug_sentence,padding='max_length', max_length = 256, 
#                     truncation=True, return_tensors="pt")
# encoder = Encoder()
# encoder1 = Encoder1()
# embed_sentence_input= encoder(sentence_input['input_ids'],sentence_input['attention_mask'])
# embed_aug_sentence_input= encoder1(aug_sentence_input['input_ids'],aug_sentence_input['attention_mask'])


# # print(embed_sentence_input)
# # print(embed_aug_sentence_input)

# print((embed_sentence_input-embed_aug_sentence_input).sum())
# # if embed_sentence_input-embed_aug_sentence_input==0:
# #     print(3)
# # else: 
# #     print(1)




## Loss

In [19]:
def unique(x, dim=None):
    """Unique elements of x and indices of those unique elements
    https://github.com/pytorch/pytorch/issues/36748#issuecomment-619514810

    e.g.

    unique(tensor([
        [1, 2, 3],
        [1, 2, 4],
        [1, 2, 3],
        [1, 2, 5]
    ]), dim=0)
    => (tensor([[1, 2, 3],
                [1, 2, 4],
                [1, 2, 5]]),
        tensor([0, 1, 3]))
    """
    unique, inverse = torch.unique(
        x, sorted=True, return_inverse=True, dim=dim)
    perm = torch.arange(inverse.size(0), dtype=inverse.dtype,
                        device=inverse.device)
    inverse, perm = inverse.flip([0]), perm.flip([0])
    return unique, inverse.new_empty(unique.size(0)).scatter_(0, inverse, perm)


class HMLC(nn.Module):
    def __init__(self, temperature=0.07,
                 base_temperature=0.07, layer_penalty=None, loss_type='hmce'):
        super(HMLC, self).__init__()
        self.temperature = temperature
        self.base_temperature = base_temperature
        if not layer_penalty:
            self.layer_penalty = self.pow_2
        else:
            self.layer_penalty = layer_penalty
        self.sup_con_loss = SupConLoss(temperature)
        self.loss_type = loss_type

    def pow_2(self, value):
        return torch.pow(2, value)

    def forward(self, features, labels):
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))
        mask = torch.ones(labels.shape).to(device)


        cumulative_loss = torch.tensor(0.0).to(device)
        max_loss_lower_layer = torch.tensor(float('-inf'))

        # original
        for l in range(0,labels.shape[1]):
        # for l in range(0,labels.shape[1]):
            mask[:, labels.shape[1]-l:] = 0
            layer_labels = labels * mask
            # print(layer_labels.shape)
            # bsz,1 
            # mask_labels = torch.stack([torch.all(torch.eq(layer_labels[i], layer_labels), dim=1)
            #                            for i in range(layer_labels.shape[0])]).type(torch.uint8).to(device)
            mask_labels = torch.stack([torch.all(torch.eq(layer_labels[i], layer_labels), dim=1)
                                for i in range(layer_labels.shape[0])]).type(torch.uint8).to(device)
                                
            # print(mask_labels.shape) # (1,1)
            # print(mask_labels)
            layer_loss = self.sup_con_loss(features, mask=mask_labels)
            # layer_loss = self.sup_con_loss(features)
            if self.loss_type == 'hmc':
                cumulative_loss += self.layer_penalty(torch.tensor(
                  1/(l)).type(torch.float)) * layer_loss
            
            elif self.loss_type == 'hce':
                layer_loss = torch.max(max_loss_lower_layer.to(layer_loss.device), layer_loss)
                cumulative_loss += layer_loss
            
            elif self.loss_type == 'hmce':
                layer_loss = torch.max(max_loss_lower_layer.to(layer_loss.device), layer_loss)
                cumulative_loss += self.layer_penalty(torch.tensor(
                    1/(l+1)).type(torch.float)) * layer_loss
            else:
                raise NotImplementedError('Unknown loss')
            
            _, unique_indices = unique(layer_labels, dim=0)
            
            max_loss_lower_layer = torch.max(
                max_loss_lower_layer.to(layer_loss.device), layer_loss)
            
            labels = labels[unique_indices]
            
            mask = mask[unique_indices]
            
            features = features[unique_indices]
        
        return cumulative_loss / labels.shape[1]
        
class SupConLoss(nn.Module):
    """Supervised Contrastive Learning: https://arxiv.org/pdf/2004.11362.pdf.
    It also supports the unsupervised contrastive loss in SimCLR"""
    def __init__(self, temperature=0.07, contrast_mode='all',
                 base_temperature=0.07):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode
        self.base_temperature = base_temperature

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric. (to indicate positive, negative sample)
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]

        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        # In the bsz, similarity with anchor
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()
        
        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)

        # mask-out self-contrast cases
        logits_mask = torch.scatter(torch.ones_like(mask),1,torch.arange(batch_size * anchor_count).view(-1, 1).to(device),0)

        mask = mask * logits_mask
        
        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))


        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)

        # print(mask.sum(1))
        # print(mean_log_prob_pos)

        # loss
        loss = - (self.temperature / self.base_temperature) * mean_log_prob_pos
        loss = loss.view(anchor_count, batch_size).mean()

        # print(loss)

        return loss

## Train

### Encoder Train

In [20]:
def set_model():
    model = Encoder()
    criterion = HMLC(0.07,'hmce',layer_penalty=torch.exp)
    criterion = criterion.cuda()
    return model, criterion

In [21]:
def main_worker():
    model, criterion = set_model()

In [22]:
# def save_checkpoint(epoch):
#     '''
#     save Model Checkpoint 
#     '''
#     model_folder = "experiment/checkpoint/"
#     model_out_path = model_folder + "epoch_{}.pth".format(epoch)
#     if not os.path.exists(model_folder):
#         os.makedirs(model_folder)
#     torch.save(model.state_dict(), model_out_path)
#     print("===> Checkpoint saved to {}".format(model_out_path))

### Encoder Train

In [23]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def train(encoder, aug_encoder, train_data, val_data, learning_rate, epochs, batch_size):
    # free_gpu_cache() 
    criterion = HMLC(temperature=temp, loss_type=loss, layer_penalty=torch.exp)
    encoder_optimizer = Adam(encoder.parameters(), lr = learning_rate, weight_decay=0.05)
    aug_encoder_optimizer = Adam(aug_encoder.parameters(), lr = learning_rate, weight_decay=0.05)

    train,val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val,batch_size=batch_size,shuffle=True)
    
    
    if torch.cuda.is_available():
        encoder = encoder.to(device)
        aug_encoder = aug_encoder.to(device)
        criterion = criterion.cuda()

    for epoch in range(epochs):
        for train_input, train_category, train_label  in tqdm(train_dataloader):
            encoder.train()
            aug_encoder.train()
             
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)
            train_label = train_label.unsqueeze(1).to(device)

            feature = encoder(input_id, mask)
            aug_feature = aug_encoder(input_id, mask)
            
            # features : original data features
            # aug_features : augmentation train data features
            f1 = feature
            f2 = aug_feature

            features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
            
            representation_loss = criterion(features,train_label)

            encoder_optimizer.zero_grad()
            aug_encoder_optimizer.zero_grad()

            representation_loss.requires_grad_(True)
            representation_loss.backward()
            
            encoder_optimizer.step()
            aug_encoder_optimizer.step()

        
        if epoch%10 == 0:
            output_file = save_folder + '/checkpoint_{:04d}.pth.tar'.format(epoch)
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': encoder,
                'state_dict': encoder.state_dict(),
                'optimizer': encoder.state_dict(), 
            }, is_best=False, filename=output_file)

            aug_output_file = save_folder + '/checkpoint_{:04d}_aug.pth.tar'.format(epoch)
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': aug_encoder,
                'state_dict': aug_encoder.state_dict(),
                'optimizer': aug_encoder.state_dict(), 
            }, is_best=False, filename=aug_output_file)

In [24]:
EPOCHS = 10
encodr = Encoder()
aug_encoder = aug_Encoder()

LR = 1e-5
batch_size = 16

train(encodr, aug_encoder, train_data, valid_data, LR, EPOCHS, batch_size)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weig

tensor([[1., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]], device='cuda:0')
tensor([[0., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.]], device='cuda:0')


  0%|          | 2/3227 [00:01<42:47,  1.26it/s]  

tensor([[1., 0., 0.,  ..., 0., 1., 0.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [1., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')
tensor([[0., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.]], device='cuda:0')


  0%|          | 3/3227 [00:02<30:08,  1.78it/s]

...

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 1., 0., 1.]], device='cuda:0')
tensor([[0., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.]], device='cuda:0')


  1%|          | 40/3227 [00:12<14:55,  3.56it/s]

tensor([[1., 0., 1.,  ..., 0., 0., 1.],
        [0., 1., 0.,  ..., 1., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.],
        ...,
        [0., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 1.,  ..., 0., 0., 1.]], device='cuda:0')
tensor([[0., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.]], device='cuda:0')


  1%|          | 40/3227 [00:12<16:47,  3.16it/s]


KeyboardInterrupt: 